In this notebook, the first part of the assignment for course 9 for Peer review, the focus is on building the code to scrape the 
wikipedia page with the following link: 'https://en.wikipedia.org/wiki/List-of-postal-codes-of-Canada:M' 

In [1]:
#Needed libraries
from bs4 import BeautifulSoup # A popular webscraping libraries among many others
import pandas as pd # provides a structure for data analysis
import requests # sends request to retrieve response from web sites

In [2]:
url='https://en.wikipedia.org/wiki/List of postal codes of Canada:M'
lopcoc=requests.get(url)

In [3]:
#create a soup object to retrieve html elements
soup=BeautifulSoup(lopcoc.text)

In [ ]:
all_links=soup.find_all('a')
for link in all_links:
    print(link.get('href'))

In [2]:
#Manual inspection of all anchor tags(a) points to this link as possible external source
url="https://en.wikipedia.org/w/index.php?search=List+of+postal+codes+of+Canada%3AM&title=Special%3ASearch&fulltext=1"
resp=requests.get(url)

In [3]:
#create a soup object to retrieve html elments after parsing by BeautifulSoup
soup=BeautifulSoup(resp.text)

In [ ]:
#provides a nice nested structure of html elements
print(soup.prettify())

In [4]:
#further inspection of the parsed html page points to this possible container holding the right link
container_of_interest=soup.find(class_="searchresults mw-searchresults-has-iw")

In [ ]:
#fetch all 'a' tags for manual inspection to finda clue to possible link to the data table
soup.find_all('a')

In [ ]:
#Inspection of all links returned gives an indication of the right link with position index for all post codes
#in Canada starting from pos=0 for postal codes beginning with M (Toronto), and index in the list is 10
soup.find_all('a')[10]

In [6]:
#This link was used to start a new web scraping as in the above
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
resp=requests.get(url)

In [7]:
soup=BeautifulSoup(resp.text)

In [ ]:
print(soup.prettify())

In [8]:
#Fortunately, there was only one table returned from this code
table=soup.findAll('table')

Getting table headers cell information in HTML element(denoted by the tags), which is found in table row element(tr) 
of a table header element(th) of a table element(tb). 
These elements can be fetched using BeautifulSoup object soup

In [ ]:
soup.findAll('tr', limit=1)

In [ ]:
soup.findAll('th')

In [11]:
#we skip the header colomns rows by setting the limit value=1
column_headers=[th.getText() for th in soup.findAll('tr', limit=1)]

In [12]:
#is found in 30 rows starting from index location 0
data_rows=soup.findAll('tr')[0:30]

In [13]:
#to retrieve data from all the rows we need to iterate over td in the rows
lopcot_data=[[td.getText() for td in data_rows[i].findAll('td')]
            for i in range(len(data_rows))]

In [17]:
#after retrieving data we create a data passing data rows and data stored in the variable lopcot_data as arguments
#and include column names
df=pd.DataFrame(lopcot_data, columns=['PostalCode', 'Borough', 'Neighborhood'])

In [21]:
# We clean the row data with special characters such as the new line('\n') at the end the Neighborhood
#column values
df['Neighborhood']=df['Neighborhood'].str.replace('\n', '')

In [22]:
df.head(30)

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned


In [26]:
#it is handy to manually create a new dataframe after conmining posstal codes. and neighborhoods as required by the assignment
df1=pd.DataFrame({'PostalCode':['M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M9A',
                               'M1B', 'M3B', 'M4B', 'M5B', 'M6B', 'M9B', 
                               'M1C'],
                 'Borough':['North York', 'North York', 'DownTown Toronto', 
                            'North York', "Queen's Park", 'Etobicoke', 'Scarborough',
                           'North York', 'East York', 'DownTown Toronto', 'North York',
                           'Etobicoke', 'Scarborough'],
                 'Neighborhood':['Parkhoods', 'Victoria Village', 'Harbourfront,Regent Park',
                                 'Lawrence Height,Lawrence Manor', "Queen's Park", 'Islington Avenue',
                                'Rouge,Malvern', 'Don Hills North', 'Woodbine Gardens,Parkview Hill',
                                'Ryerson,Garden District', 'Glencairn', 
                                 'Cloverdale,Islington,Martin Grove,Princess Gardens,West Dean Park',
                                'Highland Creek,Rouge Hill']}, columns=['PostalCode', 'Borough', 'Neighborhood'])
df1

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkhoods
1,M4A,North York,Victoria Village
2,M5A,DownTown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Height,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Hills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,DownTown Toronto,"Ryerson,Garden District"


In [27]:
df1.shape

(13, 3)